In [ ]:

####Import Library
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense,ZeroPadding2D, Activation, BatchNormalization
from keras.callbacks import CSVLogger, ReduceLROnPlateau, EarlyStopping
import glob
import cv2
from keras import optimizers
from keras.callbacks import ModelCheckpoint




###Define Constants
FAST_RUN = False
IMAGE_WIDTH=224
IMAGE_HEIGHT=224
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3
batch_size = 128
epochs = 8



In [ ]:
pip install split-folders


In [ ]:


import split_folders
split_folders.ratio('../input/plantvillage-dataset/color', output="output", seed=1337, ratio=(.7, .15,.15))


train_data_directory  = "./output/train"
validation_data_directory  = "./output/val"
test_data_directory="./output/test"
num_of_classes_on_trainDataset=print(len(glob.glob(train_data_directory +"/*")))
num_of_classes_on_valDataset=print(len(glob.glob(validation_data_directory+"/*")))
num_of_classes_on_testDataset=print(len(glob.glob(test_data_directory+"/*")))





In [ ]:


##Image Preprocessing
train_datagenerator = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   rotation_range=40,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
valid_datagenerator = ImageDataGenerator(rescale=1./255)
training_set = train_datagenerator.flow_from_directory(train_data_directory ,
                                                 target_size=IMAGE_SIZE,
                                                 batch_size=batch_size,
                                                 class_mode='categorical')

valid_set = valid_datagenerator.flow_from_directory(validation_data_directory,
                                            target_size=IMAGE_SIZE,
                                            batch_size=batch_size,
                                            class_mode='categorical')

test_datagenerator = ImageDataGenerator(rescale=1./255)
test_set = test_datagenerator.flow_from_directory(
    test_data_directory, 
    class_mode='categorical',
    target_size=IMAGE_SIZE,
    batch_size=batch_size,
    shuffle=False
)


number_of_classes= print(len(glob.glob(train_data_directory +"/*")))
number_of_train_images=training_set.samples
print(number_of_train_images)
number_of_val_images=valid_set.samples
print(number_of_val_images)
number_of_test_images=test_set.samples
print(number_of_test_images)



print(training_set.class_indices)


list_of_classes = list(training_set.class_indices.keys())
print(list_of_classes)




##See Random sample image
sample_type= random.choice(list_of_classes)
print(sample_type)

sample_random_img= random.choice(os.listdir(train_data_directory+"/"+sample_type))
print(sample_random_img)

image=load_img(train_data_directory+"/"+sample_type+"/"+sample_random_img)
plt.imshow(image)

In [ ]:
import tensorflow as tf
model=tf.keras.applications.VGG16(
    include_top=False,
    input_shape=(224, 224, 3),
    weights="imagenet")
   



In [ ]:

print(model.summary())

In [ ]:

from keras.applications.vgg16 import VGG16
from keras.utils.vis_utils import plot_model

plot_model(model, to_file='vgg.png')

In [ ]:
for layer in model.layers:
    layer.trainable = False
    
model.get_layer('block5_conv1').trainable = True
model.get_layer('block5_conv2').trainable = True
model.get_layer('block5_conv3').trainable = True
model.get_layer('block5_pool').trainable = True


In [ ]:
NewModel=Sequential()
NewModel.add(model)

NewModel.add(Flatten())
NewModel.add(Dense(units=4096,activation="relu"))
NewModel.add(Dropout(0.25))
NewModel.add(BatchNormalization())
#L15
NewModel.add(Dense(units=4096,activation="relu"))
NewModel.add(Dropout(0.25))
NewModel.add(BatchNormalization())


#L16
NewModel.add(Dense(units=38, activation="softmax"))
NewModel.summary()

In [ ]:
import tensorflow as tf
opt = tf.keras.optimizers.Adam(lr = 1e-4, decay = 1e-6)
NewModel.compile(loss="categorical_crossentropy", optimizer = opt,metrics = ["accuracy"])




In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau ,ModelCheckpoint

checkpoint = ModelCheckpoint("VGG_2_Pre.hdf5",  monitor = 'accuracy', verbose = 1, save_best_only = True, save_weights_only = False, mode = 'auto', period = 1)
history_train= NewModel.fit_generator(training_set,
                         steps_per_epoch=number_of_train_images//batch_size,
                         validation_data=valid_set,
                         epochs=epochs,
                         validation_steps=number_of_val_images//batch_size,
                         callbacks=[checkpoint],
                          verbose=1
                         )

In [ ]:
filepath="VGG_2_Pre.h5"
NewModel.save(filepath)

In [ ]:
loss, acc = NewModel.evaluate_generator(test_set, steps=number_of_test_images, verbose=1,workers=3)
print('Testing data   -> loss: %.3f, acc: %.3f' % (loss, acc))





In [ ]:

import matplotlib.pyplot as plt

acc = history_train.history['accuracy']
val_acc = history_train.history['val_accuracy']
loss = history_train.history['loss']
val_loss = history_train.history['val_loss']

epochs = range(len(acc))
#Accuracy
xmin = 0.0
xmax = 8.0
ymin = 0.30
ymax = 1.0
axes = plt.gca()
axes.set_xlim([xmin,xmax])
axes.set_ylim([ymin,ymax])

plt.plot(epochs, acc, 'r', label = 'Training accuracy')
plt.plot(epochs, val_acc, 'b', label = 'Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
#Loss
xmin = 0.0
xmax = 8.0
ymin = 0.0
ymax = 1
axes = plt.gca()
axes.set_xlim([xmin,xmax])
axes.set_ylim([ymin,ymax])


plt.plot(epochs, loss, 'r', label = 'Training Loss')
plt.plot(epochs, val_loss, 'b', label = 'Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()




In [ ]:


from keras.preprocessing import image
sample_type= random.choice(list_of_classes)
print(" original filename " + sample_type)

sample_random_img= random.choice(os.listdir(test_data_directory+"/"+sample_type))


new_image=load_img(test_data_directory+"/"+sample_type+"/"+sample_random_img ,target_size=(227, 227))


img = image.img_to_array(new_image)
img = np.expand_dims(img, axis=0)
img = img/255

print("Following is our prediction:")
prediction = NewModel.predict(img)
d = prediction.flatten()
j = d.max()
for index,item in enumerate(d):
    if item == j:
        class_name = list_of_classes[index]
    

img_prob =NewModel.predict_proba(img)
print(img_prob.max() )

plt.figure(figsize = (4,4))
plt.imshow(new_image)
plt.axis('off')
plt.title(class_name)
plt.show()